## Explore

This notebook is running in a `docker container` and being accessed from the docker host, `volumes are mapped` which means any changes here will change the file in the rpository

In [1]:
import os
os.chdir("..")

In [2]:
from src import create_app
from src.database import db
from src.models.phrase import Phrase
from src.models.language import Language
from src.models.rating import Rating
from src.models.phraseselection import PhraseSelection

In [3]:
app = create_app(test=True)

In [4]:
with app.app_context():
    db.drop_all()

In [4]:
# write data to db
mock_phraseselection_data_1 = {"userid": "1", "phraseid": 1}
mock_phraseselection_data_4 = {"userid": "2", "phraseid": 4}

mock_phrase_data_1 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 1", "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_2 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 2", "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_3 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 3" , "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_4 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 4", "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_5 = {"languageid": 2, "userid": "2",
                              "l1": "l1 phrase 4", "l2": "l2 phrase", "category": "restaurant"}


with app.app_context():
    for mock_phrase_selection_data in [mock_phraseselection_data_1, mock_phraseselection_data_4]:
            db.session.add(PhraseSelection(userid=mock_phrase_selection_data["userid"],
                                   phraseid=mock_phrase_selection_data["phraseid"]))
            db.session.commit()

    for mock_phrase_data in [mock_phrase_data_1, mock_phrase_data_2, mock_phrase_data_3, mock_phrase_data_4, mock_phrase_data_5]:

        db.session.add(Phrase(languageid=mock_phrase_data["languageid"],
                              userid=mock_phrase_data["userid"],
                              l1=mock_phrase_data["l1"],
                              l2=mock_phrase_data["l2"],
                              category=mock_phrase_data["category"]))
        db.session.commit()




In [5]:
with app.app_context():
    phrases = db.session.query(Phrase).all()
    phraseselections = db.session.query(PhraseSelection).all()

In [6]:
[c.toDict() for c in phrases]

[{'phraseid': 1,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 1',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 2,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 2',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 3,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 3',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 4,
  'userid': '1',
  'languageid': 2,
  'l1': 'l1 phrase 4',
  'l2': 'l2 phrase',
  'category': 'restaurant'},
 {'phraseid': 5,
  'userid': '2',
  'languageid': 2,
  'l1': 'l1 phrase 4',
  'l2': 'l2 phrase',
  'category': 'restaurant'}]

In [7]:
[c.toDict() for c in phraseselections]

[{'phraseselectionid': 1, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 2, 'phraseid': 4, 'userid': '2'},
 {'phraseselectionid': 3, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 4, 'phraseid': 4, 'userid': '2'},
 {'phraseselectionid': 5, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 6, 'phraseid': 4, 'userid': '2'},
 {'phraseselectionid': 7, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 8, 'phraseid': 4, 'userid': '2'},
 {'phraseselectionid': 9, 'phraseid': 1, 'userid': '1'},
 {'phraseselectionid': 10, 'phraseid': 4, 'userid': '2'}]

In [18]:
with app.app_context():
    phrase_selection_userids = PhraseSelection.query.filter(PhraseSelection.userid =="1").all()
    phrase_selection_userids = [p.userid for p in phrase_selection_userids]
    db_response =db.session.query(Phrase).\
        filter(Phrase.category == "restaurant").\
          filter(Phrase.userid.not_in(phrase_selection_userids)).all()

In [20]:
[c.toDict() for c in db_response]

[{'phraseid': 5,
  'userid': '2',
  'languageid': 2,
  'l1': 'l1 phrase 4',
  'l2': 'l2 phrase',
  'category': 'restaurant'}]